# Add reactionSMILES to VL
We add atom-mapped reactionSMILES to the virtual library.
We only do this for VL members where type == 'A'.

In [ ]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import pandas as pd

from src.util.db_utils import SynFermDatabaseConnection
from src.library_design.reaction_generator import SFReactionGenerator

In [ ]:
gen = SFReactionGenerator()

In [ ]:
# connect to DB
con = SynFermDatabaseConnection()

In [ ]:
# n.b. we just select everything now, later filter for things that the 0D model cannot handle.
# Here we don't care whether the reaction has been seen before, we can merge with the reaction data later
res = con.con.execute("SELECT id, long_name, SMILES FROM virtuallibrary WHERE type = 'A' AND initiator_long != '4-Pyrazole002';").fetchall()
df = pd.DataFrame(res, columns=["vl_id", "long_name", "product_A_smiles"])
df.head()

In [ ]:
%%time
df[0:1000]["product_A_smiles"].apply(lambda x: gen.get_reaction_smiles(x))

In [ ]:
# expected time in min to go through the entire VL
len(df) / 1000 * 19 / 60

In [ ]:
# we wrap the generator to catch errors
def get_reaction_smiles(x):
    try:
        return gen.get_reaction_smiles(x)
    except ValueError as e:
        print(e)
        print(x)
        return None
        

In [ ]:
# first we need to generate the reactionSMILES. This will take a moment 
# (to be precise I expect it to take 85 min on my MBA M1)
gen = SFReactionGenerator()
df["reaction_smiles_atom_mapped"] = df["product_A_smiles"].apply(lambda x: get_reaction_smiles(x))
df.head()

In [ ]:
# save this
with con.con:
    con.con.executemany("UPDATE virtuallibrary SET reaction_smiles_atom_mapped = ? WHERE id = ?;", df[["reaction_smiles_atom_mapped", "vl_id"]].values)